In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

import warnings
warnings.filterwarnings("ignore")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
Accidents = pd.read_csv('/content/drive/MyDrive/Accidents0515.csv',index_col='Accident_Index')
Casualities = pd.read_csv('/content/drive/MyDrive/Casualties0515.csv',error_bad_lines=False,index_col='Accident_Index',warn_bad_lines=False)
Vehicles = pd.read_csv('/content/drive/MyDrive/Vehicles0515.csv',error_bad_lines=False,index_col='Accident_Index',warn_bad_lines=False)


Select the data of 2014 year

In [ ]:
# Accidents.info()
Accidents['Date'] = pd.to_datetime(Accidents['Date'])
NewAccidents=Accidents[(Accidents['Date'].dt.year ==2014)]
NewAccidents=NewAccidents.drop(["Location_Easting_OSGR","Location_Northing_OSGR","2nd_Road_Class"
            ,"2nd_Road_Number","Pedestrian_Crossing-Human_Control","Carriageway_Hazards",
            "Did_Police_Officer_Attend_Scene_of_Accident","LSOA_of_Accident_Location"] , axis='columns')
NewAccidents.info()

<class 'pandas.core.frame.DataFrame'>
Index: 146322 entries, 201401BS70001 to 2014984139614
Data columns (total 23 columns):
 #   Column                                   Non-Null Count   Dtype         
---  ------                                   --------------   -----         
 0   Longitude                                146322 non-null  float64       
 1   Latitude                                 146322 non-null  float64       
 2   Police_Force                             146322 non-null  int64         
 3   Accident_Severity                        146322 non-null  int64         
 4   Number_of_Vehicles                       146322 non-null  int64         
 5   Number_of_Casualties                     146322 non-null  int64         
 6   Date                                     146322 non-null  datetime64[ns]
 7   Day_of_Week                              146322 non-null  int64         
 8   Time                                     146322 non-null  object        
 9   Local_Author

In [ ]:
# cleaning the other accidents index and retaining only needed
NewVehicles=Vehicles[Vehicles.index.isin(NewAccidents.index)]

NewVehicles=NewVehicles.drop(["Towing_and_Articulation",
                               "Hit_Object_off_Carriageway","Was_Vehicle_Left_Hand_Drive?","Propulsion_Code","Driver_IMD_Decile","Driver_Home_Area_Type"] , axis='columns')
NewVehicles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 268527 entries, 201401BS70001 to 2014984139614
Data columns (total 15 columns):
 #   Column                            Non-Null Count   Dtype
---  ------                            --------------   -----
 0   Vehicle_Reference                 268527 non-null  int64
 1   Vehicle_Type                      268527 non-null  int64
 2   Vehicle_Manoeuvre                 268527 non-null  int64
 3   Vehicle_Location-Restricted_Lane  268527 non-null  int64
 4   Junction_Location                 268527 non-null  int64
 5   Skidding_and_Overturning          268527 non-null  int64
 6   Hit_Object_in_Carriageway         268527 non-null  int64
 7   Vehicle_Leaving_Carriageway       268527 non-null  int64
 8   1st_Point_of_Impact               268527 non-null  int64
 9   Journey_Purpose_of_Driver         268527 non-null  int64
 10  Sex_of_Driver                     268527 non-null  int64
 11  Age_of_Driver                     268527 non-null  int64
 12  Ag

In [ ]:
NewCasualities=Casualities[Casualities.index.isin(NewAccidents.index)]

NewCasualities=NewCasualities.drop(["Car_Passenger","Bus_or_Coach_Passenger","Pedestrian_Road_Maintenance_Worker",
                               "Casualty_Home_Area_Type"] , axis='columns')


In [ ]:
NewCasualities.to_csv("casualities_2011-15.csv")
NewVehicles.to_csv("vehicle_2011-15.csv")
NewAccidents.to_csv("accident_2011-15.csv")

## **This portion is about merging the data from the other table to the main table**

In [ ]:
A = pd.read_csv('/content/accident_2011-15.csv')
C = pd.read_csv('/content/casualities_2011-15.csv',error_bad_lines=False,warn_bad_lines=False)
V = pd.read_csv('/content/vehicle_2011-15.csv',error_bad_lines=False,warn_bad_lines=False)

In [ ]:
A.info()
C.info()
V.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146322 entries, 0 to 146321
Data columns (total 24 columns):
 #   Column                                   Non-Null Count   Dtype  
---  ------                                   --------------   -----  
 0   Accident_Index                           146322 non-null  object 
 1   Longitude                                146322 non-null  float64
 2   Latitude                                 146322 non-null  float64
 3   Police_Force                             146322 non-null  int64  
 4   Accident_Severity                        146322 non-null  int64  
 5   Number_of_Vehicles                       146322 non-null  int64  
 6   Number_of_Casualties                     146322 non-null  int64  
 7   Date                                     146322 non-null  object 
 8   Day_of_Week                              146322 non-null  int64  
 9   Time                                     146322 non-null  object 
 10  Local_Authority_(District)      

Repair Accident Data

In [ ]:
police_force = pd.read_csv('/content/drive/MyDrive/contextCSVs/Police_Force.csv')
df_cd = pd.merge(A, police_force, how='inner', left_on = 'Police_Force', right_on = 'code')
df_cd.drop(['Police_Force','code'],axis=1,inplace=True)
df_cd.rename(columns={'label':'police_force'}, inplace=True)



accident_severity = pd.read_csv('/content/drive/MyDrive/contextCSVs/Accident_Severity.csv')
df_cd2 = pd.merge(df_cd, accident_severity, how='inner', left_on = 'Accident_Severity', right_on = 'code')
df_cd2.drop(['Accident_Severity','code'],axis=1,inplace=True)
df_cd2.rename(columns={'label':'accident_severity'}, inplace=True)


local_authority_district = pd.read_csv('/content/drive/MyDrive/contextCSVs/Local_Authority_District.csv')
df_cd3 = pd.merge(df_cd2, local_authority_district, how='inner', left_on = 'Local_Authority_(District)', right_on = 'code')
df_cd3.drop(['Local_Authority_(District)','code'],axis=1,inplace=True)
df_cd3.rename(columns={'label':'local_authority_district'}, inplace=True)

local_authority_highway = pd.read_csv('/content/drive/MyDrive/contextCSVs/Local_Authority_Highway.csv')
df_cd4 = pd.merge(df_cd3, local_authority_highway, how='inner', left_on = 'Local_Authority_(Highway)', right_on = 'Code')
df_cd4.drop(['Local_Authority_(Highway)','Code'],axis=1,inplace=True)
df_cd4.rename(columns={'Label':'local_authority_highway'}, inplace=True)

road_class = pd.read_csv('/content/drive/MyDrive/contextCSVs/Road_Class.csv')
df_cd5 = pd.merge(df_cd4, road_class, how='inner', left_on = '1st_Road_Class', right_on = 'code')
df_cd5.drop(['1st_Road_Class','code'],axis=1,inplace=True)
df_cd5.rename(columns={'label':'road_class'}, inplace=True)

road_type = pd.read_csv('/content/drive/MyDrive/contextCSVs/Road_Type.csv')
df_cd6 = pd.merge(df_cd5, road_type, how='inner', left_on = 'Road_Type', right_on = 'code')
df_cd6.drop(['Road_Type','code'],axis=1,inplace=True)
df_cd6.rename(columns={'label':'road_type'}, inplace=True)

junction_detail = pd.read_csv('/content/drive/MyDrive/contextCSVs/Junction_Detail.csv')
df_cd7 = pd.merge(df_cd6, junction_detail, how='inner', left_on = 'Junction_Detail', right_on = 'code')
df_cd7.drop(['Junction_Detail','code'],axis=1,inplace=True)
df_cd7.rename(columns={'label':'junction_detail'}, inplace=True)

junction_control = pd.read_csv('/content/drive/MyDrive/contextCSVs/Junction_Control.csv')
df_cd8 = pd.merge(df_cd7, junction_control, how='inner', left_on = 'Junction_Control', right_on = 'code')
df_cd8.drop(['Junction_Control','code'],axis=1,inplace=True)
df_cd8.rename(columns={'label':'junction_control'}, inplace=True)

pedestrian_crossing_facility = pd.read_csv('/content/drive/MyDrive/contextCSVs/Ped_Cross_Physical.csv')
df_cd9 = pd.merge(df_cd8, pedestrian_crossing_facility, how='inner', left_on = 'Pedestrian_Crossing-Physical_Facilities', right_on = 'code')
df_cd9.drop(['Pedestrian_Crossing-Physical_Facilities','code'],axis=1,inplace=True)
df_cd9.rename(columns={'label':'pedestrian_crossing_facility'}, inplace=True)

weather_condition = pd.read_csv('/content/drive/MyDrive/contextCSVs/Weather_Conditions.csv')
df_cd10 = pd.merge(df_cd9, weather_condition, how='inner', left_on = 'Weather_Conditions', right_on = 'code')
df_cd10.drop(['Weather_Conditions','code'],axis=1,inplace=True)
df_cd10.rename(columns={'label':'weather_conditon'}, inplace=True)

light_conditon = pd.read_csv('/content/drive/MyDrive/contextCSVs/Light_Conditions.csv')
df_cd11 = pd.merge(df_cd10, light_conditon, how='inner', left_on = 'Light_Conditions', right_on = 'code')
df_cd11.drop(['Light_Conditions','code'],axis=1,inplace=True)
df_cd11.rename(columns={'label':'light_condition'}, inplace=True)


road_condition = pd.read_csv('/content/drive/MyDrive/contextCSVs/Road_Conditions.csv')
df_cd12 = pd.merge(df_cd11, road_condition, how='inner', left_on = 'Road_Surface_Conditions', right_on = 'code')
df_cd12.drop(['Road_Surface_Conditions','code'],axis=1,inplace=True)
df_cd12.rename(columns={'label':'road_conditon'}, inplace=True)

urban_rural = pd.read_csv('/content/drive/MyDrive/contextCSVs/Urban_Rural.csv')
df_cd13 = pd.merge(df_cd12, urban_rural, how='inner', left_on = 'Urban_or_Rural_Area', right_on = 'code')
df_cd13.drop(['Urban_or_Rural_Area','code'],axis=1,inplace=True)
df_cd13.rename(columns={'label':'urban_or_rural'}, inplace=True)

special_condition = pd.read_csv('/content/drive/MyDrive/contextCSVs/special-conditon.csv')
df_cd14 = pd.merge(df_cd13, special_condition, how='inner', left_on = 'Special_Conditions_at_Site', right_on = 'code')
df_cd14.drop(['Special_Conditions_at_Site','code'],axis=1,inplace=True)
df_cd14.rename(columns={'label':'special_condition'}, inplace=True)


In [ ]:
df_cd14.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146322 entries, 0 to 146321
Data columns (total 24 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Accident_Index                146322 non-null  object 
 1   Longitude                     146322 non-null  float64
 2   Latitude                      146322 non-null  float64
 3   Number_of_Vehicles            146322 non-null  int64  
 4   Number_of_Casualties          146322 non-null  int64  
 5   Date                          146322 non-null  object 
 6   Day_of_Week                   146322 non-null  int64  
 7   Time                          146322 non-null  object 
 8   1st_Road_Number               146322 non-null  int64  
 9   Speed_limit                   146322 non-null  int64  
 10  police_force                  146322 non-null  object 
 11  accident_severity             146322 non-null  object 
 12  local_authority_district      146322 non-nul

In [ ]:
df_cd14.rename(columns={'Accident_Index':'accident_id'}, inplace=True)
df_cd14.rename(columns={'Longitude':'longitude'}, inplace=True)
df_cd14.rename(columns={'Latitude':'latitude'}, inplace=True)
df_cd14.rename(columns={'Number_of_Vehicles':'number_of_vehicles'}, inplace=True)
df_cd14.rename(columns={'Number_of_Casualties':'number_of_casualties'}, inplace=True)
df_cd14.rename(columns={'Date':'date'}, inplace=True)
df_cd14.rename(columns={'Day_of_Week':'day_of_week'}, inplace=True)
df_cd14.rename(columns={'Time':'time'}, inplace=True)
df_cd14.rename(columns={'1st_Road_Number':'road_number'}, inplace=True)
df_cd14.rename(columns={'Speed_limit':'speed_limit'}, inplace=True)

df_cd14.to_csv("accident_raw_clean.csv")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 146322 entries, 0 to 146321
Data columns (total 24 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   accident_id                   146322 non-null  object 
 1   longitude                     146322 non-null  float64
 2   latitude                      146322 non-null  float64
 3   number_of_vehicles            146322 non-null  int64  
 4   number_of_casualties          146322 non-null  int64  
 5   date                          146322 non-null  object 
 6   day_of_week                   146322 non-null  int64  
 7   time                          146322 non-null  object 
 8   road_number                   146322 non-null  int64  
 9   speed_limit                   146322 non-null  int64  
 10  police_force                  146322 non-null  object 
 11  accident_severity             146322 non-null  object 
 12  local_authority_district      146322 non-nul

Repair the vehicles data to merge together

In [ ]:
V.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 268527 entries, 0 to 268526
Data columns (total 16 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   Accident_Index                    268527 non-null  object
 1   Vehicle_Reference                 268527 non-null  int64 
 2   Vehicle_Type                      268527 non-null  int64 
 3   Vehicle_Manoeuvre                 268527 non-null  int64 
 4   Vehicle_Location-Restricted_Lane  268527 non-null  int64 
 5   Junction_Location                 268527 non-null  int64 
 6   Skidding_and_Overturning          268527 non-null  int64 
 7   Hit_Object_in_Carriageway         268527 non-null  int64 
 8   Vehicle_Leaving_Carriageway       268527 non-null  int64 
 9   1st_Point_of_Impact               268527 non-null  int64 
 10  Journey_Purpose_of_Driver         268527 non-null  int64 
 11  Sex_of_Driver                     268527 non-null  int64 
 12  Ag

In [ ]:
vehicle_location = pd.read_csv('/content/drive/MyDrive/contextCSVs/Vehicle_Location.csv')
df_cd = pd.merge(V, vehicle_location, how='inner', left_on = 'Vehicle_Location-Restricted_Lane', right_on = 'code')
df_cd.drop(['Vehicle_Location-Restricted_Lane','code'],axis=1,inplace=True)
df_cd.rename(columns={'label':'vehicle_location_restricted_lane'}, inplace=True)

junction_location = pd.read_csv('/content/drive/MyDrive/contextCSVs/Junction_Location.csv')
df_cd2 = pd.merge(df_cd, junction_location, how='inner', left_on = 'Junction_Location', right_on = 'code')
df_cd2.drop(['Junction_Location','code'],axis=1,inplace=True)
df_cd2.rename(columns={'label':'junction_location'}, inplace=True)

hit_object_in_carriageway = pd.read_csv('/content/drive/MyDrive/contextCSVs/hit_object_in_carriageway.csv')
df_cd3 = pd.merge(df_cd2, hit_object_in_carriageway, how='inner', left_on = 'Hit_Object_in_Carriageway', right_on = 'code')
df_cd3.drop(['Hit_Object_in_Carriageway','code'],axis=1,inplace=True)
df_cd3.rename(columns={'label':'hit_object_in_carriageway'}, inplace=True)


point_of_contact = pd.read_csv('/content/drive/MyDrive/contextCSVs/Point_of_Impact.csv')
df_cd4 = pd.merge(df_cd3, point_of_contact, how='inner', left_on = '1st_Point_of_Impact', right_on = 'code')
df_cd4.drop(['1st_Point_of_Impact','code'],axis=1,inplace=True)
df_cd4.rename(columns={'label':'point_of_impact'}, inplace=True)

journey_purpose = pd.read_csv('/content/drive/MyDrive/contextCSVs/Journey_Purpose.csv')
df_cd5 = pd.merge(df_cd4, journey_purpose, how='inner', left_on = 'Journey_Purpose_of_Driver', right_on = 'code')
df_cd5.drop(['Journey_Purpose_of_Driver','code'],axis=1,inplace=True)
df_cd5.rename(columns={'label':'journey_purpose'}, inplace=True)

sex = pd.read_csv('/content/drive/MyDrive/contextCSVs/Sex_of_Driver.csv')
df_cd6 = pd.merge(df_cd5, sex, how='inner', left_on = 'Sex_of_Driver', right_on = 'code')
df_cd6.drop(['Sex_of_Driver','code'],axis=1,inplace=True)
df_cd6.rename(columns={'label':'sex'}, inplace=True)

age_band = pd.read_csv('/content/drive/MyDrive/contextCSVs/Age_Band.csv')
df_cd7 = pd.merge(df_cd6, age_band, how='inner', left_on = 'Age_Band_of_Driver', right_on = 'code')
df_cd7.drop(['Age_Band_of_Driver','code'],axis=1,inplace=True)
df_cd7.rename(columns={'label':'age_group'}, inplace=True)

vehicle_type = pd.read_csv('/content/drive/MyDrive/contextCSVs/Vehicle_Type.csv')
df_cd8 = pd.merge(df_cd7, vehicle_type, how='inner', left_on = 'Vehicle_Type', right_on = 'code')
df_cd8.drop(['Vehicle_Type','code'],axis=1,inplace=True)
df_cd8.rename(columns={'label':'vehicle_type'}, inplace=True)

vehicle_mano = pd.read_csv('/content/drive/MyDrive/contextCSVs/Vehicle_Manoeuvre.csv')
df_cd9 = pd.merge(df_cd8, vehicle_mano, how='inner', left_on = 'Vehicle_Manoeuvre', right_on = 'code')
df_cd9.drop(['Vehicle_Manoeuvre','code'],axis=1,inplace=True)
df_cd9.rename(columns={'label':'vehicle_manoeuvre'}, inplace=True)

skidding = pd.read_csv('/content/drive/MyDrive/contextCSVs/skidding.csv')
df_cd10 = pd.merge(df_cd9, skidding, how='inner', left_on = 'Skidding_and_Overturning', right_on = 'code')
df_cd10.drop(['Skidding_and_Overturning','code'],axis=1,inplace=True)
df_cd10.rename(columns={'label':'skidding_and_overturning'}, inplace=True)

vehicle_leaving = pd.read_csv('/content/drive/MyDrive/contextCSVs/vehicle-leaving.csv')
df_cd11 = pd.merge(df_cd10, vehicle_leaving, how='inner', left_on = 'Vehicle_Leaving_Carriageway', right_on = 'code')
df_cd11.drop(['Vehicle_Leaving_Carriageway','code'],axis=1,inplace=True)
df_cd11.rename(columns={'label':'vehicle_leaving_carriageway'}, inplace=True)



<class 'pandas.core.frame.DataFrame'>
Int64Index: 268527 entries, 0 to 268526
Data columns (total 16 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   Accident_Index                    268527 non-null  object
 1   Vehicle_Reference                 268527 non-null  int64 
 2   Age_of_Driver                     268527 non-null  int64 
 3   Engine_Capacity_(CC)              268527 non-null  int64 
 4   Age_of_Vehicle                    268527 non-null  int64 
 5   vehicle_location_restricted_lane  268527 non-null  object
 6   junction_location                 268527 non-null  object
 7   hit_object_in_carriageway         268527 non-null  object
 8   point_of_impact                   268527 non-null  object
 9   journey_purpose                   268527 non-null  object
 10  sex                               268527 non-null  object
 11  age_group                         268527 non-null  object
 12  ve

In [ ]:
df_cd11.rename(columns={'Accident_Index':'accident_id'}, inplace=True)
df_cd11.rename(columns={'Vehicle_Reference':'vehicle_id'}, inplace=True)
df_cd11.rename(columns={'Age_of_Driver':'age'}, inplace=True)
df_cd11.rename(columns={'Engine_Capacity_(CC)':'engine_capacity'}, inplace=True)
df_cd11.rename(columns={'Age_of_Vehicle':'age_of_vehicle'}, inplace=True)
df_cd11.to_csv("vehicle_raw_clean.csv")

In [ ]:
df_cd11.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 268527 entries, 0 to 268526
Data columns (total 16 columns):
 #   Column                            Non-Null Count   Dtype 
---  ------                            --------------   ----- 
 0   accident_id                       268527 non-null  object
 1   vehicle_id                        268527 non-null  int64 
 2   age                               268527 non-null  int64 
 3   engine_capacity                   268527 non-null  int64 
 4   age_of_vehicle                    268527 non-null  int64 
 5   vehicle_location_restricted_lane  268527 non-null  object
 6   junction_location                 268527 non-null  object
 7   hit_object_in_carriageway         268527 non-null  object
 8   point_of_impact                   268527 non-null  object
 9   journey_purpose                   268527 non-null  object
 10  sex                               268527 non-null  object
 11  age_group                         268527 non-null  object
 12  ve

**Repair the data for Casualty**

In [ ]:
C.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194477 entries, 0 to 194476
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   Accident_Index        194477 non-null  object
 1   Vehicle_Reference     194477 non-null  int64 
 2   Casualty_Reference    194477 non-null  int64 
 3   Casualty_Class        194477 non-null  int64 
 4   Sex_of_Casualty       194477 non-null  int64 
 5   Age_of_Casualty       194477 non-null  int64 
 6   Age_Band_of_Casualty  194477 non-null  int64 
 7   Casualty_Severity     194477 non-null  int64 
 8   Pedestrian_Location   194477 non-null  int64 
 9   Pedestrian_Movement   194477 non-null  int64 
 10  Casualty_Type         194477 non-null  int64 
dtypes: int64(10), object(1)
memory usage: 16.3+ MB


In [ ]:
casualty_class = pd.read_csv('/content/drive/MyDrive/contextCSVs/Casualty_Class.csv')
df_cd = pd.merge(C, casualty_class, how='inner', left_on = 'Casualty_Class', right_on = 'code')
df_cd.drop(['Casualty_Class','code'],axis=1,inplace=True)
df_cd.rename(columns={'label':'casualty_class'}, inplace=True)

sex = pd.read_csv('/content/drive/MyDrive/contextCSVs/Sex_of_Driver.csv')
df_cd2 = pd.merge(df_cd, sex, how='inner', left_on = 'Sex_of_Casualty', right_on = 'code')
df_cd2.drop(['Sex_of_Casualty','code'],axis=1,inplace=True)
df_cd2.rename(columns={'label':'sex'}, inplace=True)

age_group = pd.read_csv('/content/drive/MyDrive/contextCSVs/Age_Band.csv')
df_cd3 = pd.merge(df_cd2, age_group, how='inner', left_on = 'Age_Band_of_Casualty', right_on = 'code')
df_cd3.drop(['Age_Band_of_Casualty','code'],axis=1,inplace=True)
df_cd3.rename(columns={'label':'age_group'}, inplace=True)

casualty_severity = pd.read_csv('/content/drive/MyDrive/contextCSVs/Casualty_Severity.csv')
df_cd4 = pd.merge(df_cd3, casualty_severity, how='inner', left_on = 'Casualty_Severity', right_on = 'code')
df_cd4.drop(['Casualty_Severity','code'],axis=1,inplace=True)
df_cd4.rename(columns={'label':'casualty_severity'}, inplace=True)

ped_movement = pd.read_csv('/content/drive/MyDrive/contextCSVs/Ped_Movement.csv')
df_cd5 = pd.merge(df_cd4, ped_movement, how='inner', left_on = 'Pedestrian_Movement', right_on = 'code')
df_cd5.drop(['Pedestrian_Movement','code'],axis=1,inplace=True)
df_cd5.rename(columns={'label':'pedestrian_movement'}, inplace=True)

ped_loc = pd.read_csv('/content/drive/MyDrive/contextCSVs/Ped_Location.csv')
df_cd6 = pd.merge(df_cd5, ped_loc, how='inner', left_on = 'Pedestrian_Location', right_on = 'code')
df_cd6.drop(['Pedestrian_Location','code'],axis=1,inplace=True)
df_cd6.rename(columns={'label':'pedestrian_location'}, inplace=True)

casualty_type = pd.read_csv('/content/drive/MyDrive/contextCSVs/Casualty_Type.csv')
df_cd7 = pd.merge(df_cd6, casualty_type, how='inner', left_on = 'Casualty_Type', right_on = 'code')
df_cd7.drop(['Casualty_Type','code'],axis=1,inplace=True)
df_cd7.rename(columns={'label':'casualty_type'}, inplace=True)


df_cd7.rename(columns={'Accident_Index':'accident_id'}, inplace=True)
df_cd7.rename(columns={'Vehicle_Reference':'vehicle_id'}, inplace=True)
df_cd7.rename(columns={'Casualty_Reference':'casualty_id'}, inplace=True)
df_cd7.rename(columns={'Age_of_Casualty':'age'}, inplace=True)


df_cd7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194477 entries, 0 to 194476
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   accident_id          194477 non-null  object
 1   vehicle_id           194477 non-null  int64 
 2   casualty_id          194477 non-null  int64 
 3   age                  194477 non-null  int64 
 4   casualty_class       194477 non-null  object
 5   sex                  194477 non-null  object
 6   age_group            194477 non-null  object
 7   casualty_severity    194477 non-null  object
 8   pedestrian_movement  194477 non-null  object
 9   pedestrian_location  194477 non-null  object
 10  casualty_type        194477 non-null  object
dtypes: int64(3), object(8)
memory usage: 17.8+ MB


In [ ]:
df_cd7.to_csv("casualty_raw_clean.csv")

In [ ]:
df_cd7.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 194477 entries, 0 to 194476
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   accident_id          194477 non-null  object
 1   vehicle_id           194477 non-null  int64 
 2   casualty_id          194477 non-null  int64 
 3   age                  194477 non-null  int64 
 4   casualty_class       194477 non-null  object
 5   sex                  194477 non-null  object
 6   age_group            194477 non-null  object
 7   casualty_severity    194477 non-null  object
 8   pedestrian_movement  194477 non-null  object
 9   pedestrian_location  194477 non-null  object
 10  casualty_type        194477 non-null  object
dtypes: int64(3), object(8)
memory usage: 17.8+ MB
